<a href="https://colab.research.google.com/github/phanikarra15/NLP_Final_Project/blob/main/Emphasis_Selection_Phani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 2.9MB 17.1MB/s 
     |████████████████████████████████| 1.1MB 28.2MB/s 
     |████████████████████████████████| 890kB 39.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=34c46da50347ca95785b60c9a953284f19cd80d1d79e269fe608e971f72d9c76
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
from transformers import AutoTokenizer, AutoModel, AdamW
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
print(device)


cpu


In [3]:
def dataset(filename):
    with open(filename,'r') as fp:
        lines = [line.strip() for line in fp]
    return lines

In [4]:
def wordData(data):
    wordLines = data
    words = []
    probabilities = []
    wordList = []
    pos = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]
            prob = lineSplit[4]
            temp = lineSplit[5]
            words.append(word)
            probabilities.append(prob)
            pos.append(temp)
        elif not (len(empty) and []):
            wordList.append((words, pos, probabilities))
            words = []
            probabilities = []
            pos = []
    return wordList

In [5]:
def wordtest(data):
    wordLines = data
    words = []
    testWord = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]            
            words.append(word)
        elif not len(empty):
            testWord.append(words)
            words = []       
    return testWord

In [6]:
def preTokenizing(data):
    t = []
    po = []
    prob = []
    for i,j,k in data:
        text = []
        pos = []
        probs = []
        for l in i:
            text.append(l)
        for m in j:
            pos.append(m)
        for n in k:
            probs.append(n)
        t.append(text)
        po.append(pos)
        prob.append(probs)
    return t,po, prob

In [7]:
def preTokenizingtest(data):
    text = []
    for i in data:
        for j in i:
            text.append(j)
    return text

In [8]:
def sep(words):
    text = []
    for i in words:
        sent = ""
        for j in i:
            if sent == "":
                sent += j
            else:
                sent = sent + " " + j
        
        tokens = tokenizer.tokenize(sent)
        to_id = tokenizer.encode_plus(tokens,max_length= 40,pad_to_max_length = True,return_attention_mask=True,add_special_tokens=True, return_token_type_ids=False, return_tensors = "pt")
        text.append((to_id))
    return text

In [9]:
TRAINING_FILE = "train.txt"
DEV_FILE = "dev.txt"
TEST_FILE = "test_data.txt"

In [12]:
trainText = wordData(dataset(TRAINING_FILE))
testEval = wordtest(dataset(TEST_FILE))
devText = wordData(dataset(DEV_FILE))

trainWords,trainTags, trainLabels = preTokenizing(trainText)
devWords, devTags, devLabels = preTokenizing(devText)
testWords = preTokenizingtest(testEval)

tokenized_text = sep(trainWords)
tokenized_dev = sep(devWords)
#tokenized_text = DataLoader(tokenized_text, batch_size=16, shuffle=True)
#tokenized_dev = DataLoader(tokenized_dev, batch_size=16, shuffle=True)



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(tokenized_text)

In [14]:
sampleTrain = tokenized_text[:50]

In [15]:
model = AutoModel.from_pretrained("nghuyong/ernie-2.0-en")

optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
for epoch in range(1):
    for batch in sampleTrain:
        optim.zero_grad()
        outputs,hidden = model(batch["input_ids"],batch["attention_mask"])
        optim.step()

model.eval()

In [27]:
print(outputs.shape)
print(hidden.shape)

torch.Size([1, 40, 768])
torch.Size([1, 768])


In [28]:
model.config.hidden_size,model.config.num_hidden_layers,model.config.num_attention_heads,hidden.shape

(768, 12, 12, torch.Size([1, 768]))

In [36]:
print("outputs------>",outputs)

outputs------> torch.FloatTensor


In [30]:
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.n_feature = n_feature
        self.n_output = n_output
        self.n_hidden = n_hidden
        self.encoder = nn.Embedding(self.n_feature, self.n_output)
        #self.auto = AutoModel.from_pretrained('nghuyong/ernie-2.0-en')
        self.hidden = nn.Linear(self.n_feature, self.auto.config.hidden_size)   # hidden layer
        self.predict = nn.Linear(self.auto.config.hidden_size, self.n_output)   # output layer
    def forward(self, input_ids, attention_mask):
        self.encoder = self.encoder(input_ids)
        self.attention_mask = self.attention_mask
        _, pooled_output = self.auto(input_ids=encoder, attention_mask=attention_mask)
        output = self.drop(pooled_output)
        return self.out(output)
    
net = Net(n_feature=300, n_hidden=12, n_output=200)

In [31]:
model_op = net(outputs,hidden)

RuntimeError: ignored